In [31]:
import os.path
import utilities as helper
import warnings

import tensorflow as tf
from distutils.version import LooseVersion
import tests
import numpy as np
import shutil
from urllib.request import urlretrieve

In [5]:
# Hyperparameters...
KEEP_PROB = 0.75
BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 0.0001
NORM_STDDEV = 0.01

In [11]:
def load(sess, vgg_path):
    vgg_tag = 'vgg16'
    vgg_input_tensor = 'image_input:0'
    vgg_keep_prob_tensor = 'keep_prob:0'
    vgg_layer3_out_tensor = 'layer3_out:0'
    vgg_layer4_out_tensor = 'layer4_out:0'
    vgg_layer7_out_tensor = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)

    graph = tf.get_default_graph()

    input_tensor = graph.get_tensor_by_name(vgg_input_tensor)
    keep_prob_tensor = graph.get_tensor_by_name(vgg_keep_prob_tensor)
    layer3_tensor = graph.get_tensor_by_name(vgg_layer3_out_tensor)
    layer4_tensor = graph.get_tensor_by_name(vgg_layer4_out_tensor)
    layer7_tensor = graph.get_tensor_by_name(vgg_layer7_out_tensor)
    return input_tensor, keep_prob_tensor, layer3_tensor, layer4_tensor, layer7_tensor

In [12]:
# This is just to check for errors. Nothing to do with actual task. OPTIONAL.
tests.test_load_vgg(load, tf)

Tests Passed


In [8]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    kernel_initializer = tf.truncated_normal_initializer(stddev = NORM_STDDEV)
    conv_1x1 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding='same', strides=(1,1),
                                 kernel_initializer= kernel_initializer)
    decode_layer_1 = tf.layers.conv2d_transpose(conv_1x1, num_classes, 4, strides=(2,2), padding='same', kernel_initializer= kernel_initializer)
    vgg_layer4_1x1 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, padding='same', strides=(1,1),
                                 kernel_initializer= kernel_initializer)
    decode_layer_1_with_skip = tf.add(decode_layer_1, vgg_layer4_1x1)
    decode_layer_2 = tf.layers.conv2d_transpose(decode_layer_1_with_skip, num_classes, 4, strides=(2,2), padding='same', kernel_initializer= kernel_initializer)
    vgg_layer3_1x1 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, padding='same', strides=(1,1),
                                 kernel_initializer= kernel_initializer)
    decode_layer_2_with_skip = tf.add(decode_layer_2, vgg_layer3_1x1)
    decoder_output = tf.layers.conv2d_transpose(decode_layer_2_with_skip, num_classes, 16, strides=(8,8), padding='same', kernel_initializer= kernel_initializer)
    return decoder_output

In [9]:
# Again to check for errors. OPTIONAL.
tests.test_layers(layers)

Tests Passed


In [14]:
def opt(nn_last_layer, correct_label, learning_rate, num_classes):
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    labels = tf.reshape(correct_label, (-1, num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)  
    return logits, train_op, cross_entropy_loss

In [15]:
# Again to check for errors. OPTIONAL.
tests.test_optimize(opt)

Tests Passed


In [17]:
def trainnn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    sess.run(tf.global_variables_initializer())
    
    print("Training...")
    for i in range(epochs):
        print("EPOCH {} ...".format(i+1))
        j = 0
        for image, label in get_batches_fn(batch_size):
            print("Batch {} ".format(j+1))
            j += 1
            sess.run(train_op, feed_dict={input_image: image, correct_label: label, keep_prob: KEEP_PROB, learning_rate: LEARNING_RATE})

In [18]:
tests.test_train_nn(trainnn)

In [37]:
# A LIBRARY FOR SHOWING PROGRESS BARS. NOTHING TO WITH ORIGINAL TASK. JUST FOR DOWNLOADING THE VGG16 MODEL. OPTIONAL.
from tqdm import tqdm
class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

In [34]:
def download_model(data_dir):
    vgg_filename = 'vgg.zip'
    vgg_path = os.path.join(data_dir, 'vgg')
    vgg_files = [
        os.path.join(vgg_path, 'variables/variables.data-00000-of-00001'),
        os.path.join(vgg_path, 'variables/variables.index'),
        os.path.join(vgg_path, 'saved_model.pb')]

    missing_vgg_files = [vgg_file for vgg_file in vgg_files if not os.path.exists(vgg_file)]
    if missing_vgg_files:
        if os.path.exists(vgg_path):
            shutil.rmtree(vgg_path)
        os.makedirs(vgg_path)

        print('Downloading pre-trained vgg model...')
        with DLProgress(unit='B', unit_scale=True, miniters=1) as pbar:
            urlretrieve(
                'https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/vgg.zip',
                os.path.join(vgg_path, vgg_filename),
                pbar.hook)

        print('Extracting model...')
        zip_ref = zipfile.ZipFile(os.path.join(vgg_path, vgg_filename), 'r')
        zip_ref.extractall(data_dir)
        zip_ref.close()
        os.remove(os.path.join(vgg_path, vgg_filename))

In [33]:
# RUNNING THIS WILL CAUSE A DOWNLOAD OF 997 MB VGG16 PRETRAINED MODEL. WHICH IS REQUIRED TO TRAIN OUR MODEL.
download_model('./data')
# I AM INTERRUPTING THIS FOR NOW AS IT WILL TAKE AROUND 11 HOURS IN MY SLOW WI-FI !!

  0%|▏                                                                                                                          | 1.49M/997M [01:04<11:53:27, 23.3KB/s]


KeyboardInterrupt: 

In [19]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    video_dir = './video'
    proc_vid_dir = './processed_video'
    tests.test_for_kitti_dataset(data_dir)

    epochs = EPOCHS
    batch_size = BATCH_SIZE

    correct_label = tf.placeholder(tf.float32, (None,image_shape[0],image_shape[1],2))
    learning_rate = tf.placeholder(tf.float32, (None))

    with tf.Session() as sess:
        vgg_path = os.path.join(data_dir, 'vgg')
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)
        # TO LOAD THE MODEL.. CALL THE LOAD METHOD.
        input_tensor, keep_prob_tensor, layer3_tensor, layer4_tensor, layer7_tensor = load(sess, vgg_path)
        # TO CREATE FOR FCN (FULLY CONVOLUTIONAL NETWORK) AND GRAB THE LAST LAYER..
        nn_last_layer = layers(layer3_tensor,layer4_tensor,layer7_tensor, num_classes)
        # TO CREATE THE OPTIMIZER.. CALL THE OPT METHOD..
        logits, train_op, cross_entropy_loss =  opt(nn_last_layer, correct_label, learning_rate, num_classes)
        trainnn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_tensor
                 , correct_label, keep_prob_tensor, learning_rate)
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob_tensor, input_tensor)

In [38]:
# RUNNING THIS WILL DO THE FOLLOWING.
# LOAD MODEL WHICH WAS DOWNLOADED BY "download_model" METHOD AND STORED IN DATA FOLDER.
# LOAD THE REQUIRED LAYERS BY "layers" METHOD..
# CREATE OPTIMIZER BY "opt" METHOD..
# TRAINING THE MODEL BY "trainnn" METHOD..
run()